In [1]:
from pathlib import Path
import optuna
from fastai.tabular.core import cont_cat_split
import pandas as pd
from sklearn import compose, impute, pipeline, preprocessing, model_selection, svm
import numpy as np
from sklearn import metrics
from fastai.tabular.all import *

path = Path("./data")
output_path = Path("./output")

In [2]:
def balanced_log_loss(y_true, y_pred):
    nc = np.bincount(y_true)
    return metrics.log_loss(y_true, y_pred, sample_weight=1 / nc[y_true], eps=1e-15)

In [3]:
def balanced_log_loss_tensor(output: Tensor, target: Tensor):
    y_true = target.flatten().cpu().detach().numpy()
    y_pred = output.cpu().detach().numpy()

    try:
        loss_value = balanced_log_loss(y_true, y_pred)
    except Exception:
        return torch.tensor(0.0, dtype=torch.float32, device=output.device, requires_grad=True)
    return torch.tensor(loss_value, dtype=torch.float32, device=output.device, requires_grad=True)

In [4]:
class BalancedLogLoss:
    y_int = True

    def __init__(self, *args, **kwargs):
        store_attr()
        
    def __call__(self, inp, targ, **kwargs):
        return balanced_log_loss_tensor(inp, targ, **kwargs)

    def activation(self, out: Tensor) -> Tensor:
        return F.softmax(out, dim=-1)

    def decodes(self, out: Tensor) -> Tensor:
        return out.argmax(dim=-1)

In [5]:
from collections import defaultdict

import numpy as np
import pandas as pd
from scipy import stats
from sklearn import compose, ensemble, impute, pipeline, preprocessing, tree

def get_preprocess_pipeline(df, cont_cols, cat_cols, drop_cols):
    """
    Returns a pipeline that performs the following transformations:
    * Standard scaling
    * Log transformation
    * Reciprocal transformation
    * Box-Cox transformation
    * Yeo-Johnson transformation
    * Categorical imputing
    * Semi-constant feature binarization

    Based on the EDA from https://www.kaggle.com/code/mateuszk013/icr-eda-balanced-learning-with-lgbm-xgb/notebook

    :param df: The dataframe to be transformed.
    :type df: pandas.DataFrame
    :param cont_names: The names of the continuous variables.
    :type cont_names: list of str
    :param dep_vars: The names of the dependent variables.
    :type dep_vars: list of str
    """

    # Identify columns that doesn't follow a normal distribution
    # find an appropriate transformation for them to follow a normal distribution
    r2_scores = defaultdict(tuple)

    for feature in cont_cols:
        orig = df[feature].dropna()
        _, (*_, R_orig) = stats.probplot(orig, rvalue=True)
        _, (*_, R_log) = stats.probplot(np.log(orig), rvalue=True)
        _, (*_, R_sqrt) = stats.probplot(np.sqrt(orig), rvalue=True)
        _, (*_, R_reci) = stats.probplot(np.reciprocal(orig), rvalue=True)
        _, (*_, R_boxcox) = stats.probplot(stats.boxcox(orig)[0], rvalue=True)
        _, (*_, R_yeojohn) = stats.probplot(stats.yeojohnson(orig)[0], rvalue=True)
        r2_scores[feature] = (
            R_orig * R_orig,
            R_log * R_log,
            R_sqrt * R_sqrt,
            R_reci * R_reci,
            R_boxcox * R_boxcox,
            R_yeojohn * R_yeojohn,
        )

    r2_scores = pd.DataFrame(
        r2_scores,
        index=("Original", "Log", "Sqrt", "Reciprocal", "BoxCox", "YeoJohnson"),
    ).T

    r2_scores["Winner"] = r2_scores.idxmax(axis=1)

    # Identify columns to be transformed
    no_transform_cols = r2_scores.query("Winner == 'Original'").index
    log_transform_cols = r2_scores.query("Winner == 'Log'").index
    reciprocal_transform_cols = r2_scores.query("Winner == 'Reciprocal'").index
    boxcox_transform_cols = r2_scores.query("Winner == 'BoxCox'").index
    yeojohnson_transform_cols = r2_scores.query("Winner == 'YeoJohnson'").index

    # Identify columns that are constant or semi-constant
    numeric_descr = df.drop(columns=drop_cols).describe().T
    semi_constant_mask = np.isclose(numeric_descr["min"], numeric_descr["50%"])
    semi_constant_descr = numeric_descr[semi_constant_mask]
    semi_const_cols_thresholds = semi_constant_descr["50%"].to_dict()

    # List of columns to be transformed
    semi_const_cols = semi_const_cols_thresholds.keys()
    no_transform_cols = no_transform_cols.drop(semi_const_cols, errors="ignore").to_list()
    log_transform_cols = log_transform_cols.drop(semi_const_cols, errors="ignore").to_list()
    reciprocal_transform_cols = reciprocal_transform_cols.drop(semi_const_cols, errors="ignore").to_list()
    boxcox_transform_cols = boxcox_transform_cols.drop(semi_const_cols, errors="ignore").to_list()
    yeojohnson_transform_cols = yeojohnson_transform_cols.drop(semi_const_cols, errors="ignore").to_list()

    # Transformations
    standard_scaling = (
        preprocessing.StandardScaler(),
        no_transform_cols,
    )
    log_transform = (
        pipeline.make_pipeline(
            preprocessing.FunctionTransformer(func=np.log, feature_names_out="one-to-one"),
            preprocessing.StandardScaler(),
        ),
        log_transform_cols,
    )
    reciprocal_transform = (
        pipeline.make_pipeline(
            preprocessing.FunctionTransformer(func=np.reciprocal, feature_names_out="one-to-one"),
            preprocessing.StandardScaler(),
        ),
        reciprocal_transform_cols,
    )
    boxcox_transform = (
        preprocessing.PowerTransformer(method="box-cox", standardize=True),
        boxcox_transform_cols,
    )
    yeojohnson_transform = (
        preprocessing.PowerTransformer(method="yeo-johnson", standardize=True),
        yeojohnson_transform_cols,
    )

    # Other transformations
    categorical_imputing = (
        pipeline.make_pipeline(
            impute.SimpleImputer(strategy="most_frequent"),
            preprocessing.OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
        ),
        cat_cols,  # type: ignore
    )
    semi_const_transforms = [
        (
            pipeline.make_pipeline(
                impute.SimpleImputer(strategy="median"),
                preprocessing.Binarizer(threshold=thresh),
            ),
            [col],
        )
        for col, thresh in semi_const_cols_thresholds.items()
    ]

    return pipeline.make_pipeline(
        compose.make_column_transformer(
            standard_scaling,
            log_transform,
            reciprocal_transform,
            boxcox_transform,
            yeojohnson_transform,
            categorical_imputing,
            *semi_const_transforms,
            remainder="drop",
            verbose_feature_names_out=False,
        ),
        impute.KNNImputer(n_neighbors=10, weights="distance"),
    ).set_output(transform="pandas")

    
# Load data
    
df = pd.read_csv(path / "train.csv", index_col="Id")
dep_vars = ["Class"]

drop_vars = ["EJ"]
df.drop(columns=drop_vars, inplace=True)

train_df, test_df = model_selection.train_test_split(df, test_size=0.4, stratify=df[dep_vars], random_state=33)

cont_names, cat_names = cont_cat_split(df, dep_var=dep_vars)

preprocessor = get_preprocess_pipeline(df, cont_names, cat_names, dep_vars)

# Preprocess training data
X_pre = preprocessor.fit_transform(train_df.drop(columns=dep_vars))
train_df = pd.merge(X_pre, train_df[dep_vars], left_index=True, right_index=True)
X = train_df.drop(columns=dep_vars, errors="ignore")
y = train_df[dep_vars]

# Preprocess test data
X_test_pre = preprocessor.transform(test_df.drop(columns=dep_vars))
test_df = pd.merge(X_test_pre, test_df[dep_vars], left_index=True, right_index=True)
X_test = test_df.drop(columns=dep_vars, errors="ignore")
y_test = test_df[dep_vars]

In [6]:
# # Resample
# from imblearn.over_sampling import SMOTE


# def resample(X, y):
#     sampler = SMOTE()
#     X_res, y_res = sampler.fit_resample(X, y)
#     return X_res, y_res

In [7]:
# X_res, y_res = resample(X, y)
# train_df_res = pd.merge(X_res, y_res, left_index=True, right_index=True)


# Optimize

In [8]:
layers_map = {
    0: [2048, 1024, 512],
    1: [2048, 1024, 512, 256],
    2: [2048, 1024, 512, 256, 128],
    3: [2048, 1024, 512, 256, 128],
    4: [2048, 1024, 512, 256, 128, 64],
    5: [2048, 1024, 512, 256, 128, 64, 32],
    6: [2048, 1024, 512, 256, 128, 64, 32, 16],
    7: [512, 256, 128, 64, 32, 16],
    8: [256, 128, 64, 32, 16],
    9: [128, 64, 32, 16],
    10: [64, 32, 32, 16, 8],
}

In [9]:
def objective(trial):
    bs = trial.suggest_int("bs", 8, 128, log=True)
    epochs = trial.suggest_int("epochs", 1, 64)
    layers_choice = trial.suggest_categorical("layers_choice", layers_map.keys())
    layers = layers_map[layers_choice]
    trial.set_user_attr("layers", layers)

    config = tabular_config(
        ps=trial.suggest_float("ps", 0.01, 0.5, log=True),
        use_bn=trial.suggest_categorical("use_bn", [True, False]),
        bn_final=trial.suggest_categorical("bn_final", [True, False]),
        bn_cont=trial.suggest_categorical("bn_cont", [True, False]),
        lin_first=trial.suggest_categorical("lin_first", [True, False]),
    )

    dls = TabularDataLoaders.from_df(
        train_df,
        y_names=dep_vars,
        y_block=CategoryBlock,
        splitter=RandomSplitter(seed=42),
        bs=bs,
        splits=TrainTestSplitter(
            test_size=0.2,
            stratify=train_df[dep_vars],
        ),
    )

    model = tabular_learner(
        dls,
        loss_func=FocalLossFlat(gamma=trial.suggest_float("gamma", 0.5, 5.0, step=0.5)),
        layers=layers,
        config=config,
        cbs=[
            EarlyStoppingCallback(min_delta=0.1, patience=5),
        ],
        wd=trial.suggest_float("wd", 1e-8, 1e-3, log=True),
        wd_bn_bias=trial.suggest_categorical("wd_bn_bias", [True, False]),
    )

    with model.no_logging(), model.no_bar():
        model.fit_one_cycle(epochs)

    ypreds, targs = model.get_preds()
    val_loss = balanced_log_loss_tensor(ypreds, targs).item()
    
    test_dl = dls.test_dl(test_df)
    ypreds, targs = model.get_preds(dl=test_dl)
    test_loss = balanced_log_loss_tensor(ypreds, targs).item()

    return test_loss, test_loss - val_loss

In [10]:
# optuna.delete_study(
#     study_name="fastai_no_categorical_no_resample",
#     storage="sqlite:///optuna.db",
# )

In [11]:
pruner = optuna.pruners.SuccessiveHalvingPruner()
study = optuna.create_study(
    pruner=pruner,
    directions=["minimize", "minimize"],
    study_name="fastai_no_categorical_no_resample",
    storage="sqlite:///optuna.db",
    load_if_exists=True,
)
study.optimize(objective, n_trials=250)

[I 2023-08-06 21:33:53,998] A new study created in RDB with name: fastai_no_categorical_no_resample


No improvement since epoch 0: early stopping


[I 2023-08-06 21:33:56,778] Trial 0 finished with values: [0.6455721259117126, 0.013538837432861328] and parameters: {'bn_cont': True, 'bn_final': True, 'bs': 13, 'epochs': 27, 'gamma': 3.0, 'layers_choice': 8, 'lin_first': True, 'ps': 0.0501472859002621, 'use_bn': True, 'wd': 3.382326142226763e-05, 'wd_bn_bias': False}. 


No improvement since epoch 0: early stopping


[I 2023-08-06 21:33:58,831] Trial 1 finished with values: [0.6513509154319763, -0.01633065938949585] and parameters: {'bn_cont': True, 'bn_final': False, 'bs': 9, 'epochs': 46, 'gamma': 1.0, 'layers_choice': 9, 'lin_first': True, 'ps': 0.028595987070729204, 'use_bn': True, 'wd': 5.180790609932885e-06, 'wd_bn_bias': True}. 


...

[I 2023-08-06 21:38:57,445] Trial 247 finished with values: [0.0, 0.0] and parameters: {'bn_cont': False, 'bn_final': False, 'bs': 41, 'epochs': 59, 'gamma': 0.5, 'layers_choice': 5, 'lin_first': True, 'ps': 0.027538565216866734, 'use_bn': False, 'wd': 8.862481565197366e-07, 'wd_bn_bias': False}. 


No improvement since epoch 0: early stopping


[I 2023-08-06 21:38:58,095] Trial 248 finished with values: [0.5467091202735901, -0.03391897678375244] and parameters: {'bn_cont': False, 'bn_final': False, 'bs': 36, 'epochs': 39, 'gamma': 0.5, 'layers_choice': 9, 'lin_first': True, 'ps': 0.013760141065833956, 'use_bn': True, 'wd': 5.6122453107325e-07, 'wd_bn_bias': False}. 


No improvement since epoch 2: early stopping


[I 2023-08-06 21:38:58,785] Trial 249 finished with values: [0.0, 0.0] and parameters: {'bn_cont': True, 'bn_final': False, 'bs': 49, 'epochs': 48, 'gamma': 0.5, 'layers_choice': 0, 'lin_first': True, 'ps': 0.066262339735798, 'use_bn': False, 'wd': 5.4637872455215026e-05, 'wd_bn_bias': True}. 
